<a href="https://colab.research.google.com/github/thapasya-m/2048-undo-redo-replay/blob/main/OJP_trim_event.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install obspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import obspy
from obspy.clients.fdsn import Client
from obspy.taup import TauPyModel
from obspy import UTCDateTime
from obspy.core import AttribDict
from obspy.io.sac import SACTrace
from obspy.geodetics import gps2dist_azimuth, locations2degrees
from obspy import read_inventory
import numpy as np
import os
from datetime import datetime
import calendar
import urllib
import pandas as pd

In [3]:
# Specify the file path
file_path = './CTBTO-stations.txt'

# Specify the separator used in the file
separator = '|'

# Read the text file into a DataFrame
df = pd.read_csv(file_path, sep=separator)

# Remove trailing spaces in column names
df.columns = df.columns.str.strip()

# Display the DataFrame
df.head()

,#Network,Station,Latitude,Longitude,Elevation,Sitename,StartTime,EndTime
0,IM,ABKAR,49.255600,59.943100,243.0,"Akbulak Array (ABKAR), Beam, Kazakhstan",2011-08-16T00:00:00,2599-12-31T23:59:59
1,IM,AKASG,50.701199,29.224199,160.0,"Malin Array Beam, Ukraine",1972-01-03T00:00:00,2599-12-31T23:59:59
2,IM,ARCES,69.534897,25.505800,403.0,"ARCESS Array Beam, Norway",1972-01-03T00:00:00,2599-12-31T23:59:59
3,IM,ASAR,-23.666401,133.904007,607.0,Alice Springs Array Beam,2005-06-21T00:00:00,2599-12-31T23:59:59
4,IM,ATTU,52.882099,173.164307,250.0,"Attu Island, AK, USA",1972-01-03T00:00:00,2010-04-01T19:14:56


## Gather station level info

In [4]:
new_df = pd.DataFrame(columns=[
    'station_name',
    'station_lat',
    'station_lon',
    'start_time',
    'end_time',
    'e_origin_time',
    'e_lat',
    'e_lon',
    'e_dep',
    'e_baz',
    'e_gcarc',
    'e_mag',
    'travel_time'
])

In [ ]:
# # global start and end time
# time1 = df['StartTime'].min() # df.loc[0, 'StartTime']
# time2 = df['EndTime'].max()

# Get event catalog
cat_client="IRIS"       # catalog data
client=Client(cat_client)
minmag=6.0 # minimum magnitude

# for stn in df['Station'].unique():
for index, row in df.iterrows():
  stn_time1 = row['StartTime']
  stn_time2 = row['EndTime']

  t1=UTCDateTime(stn_time1)           # start date
  t2=UTCDateTime(stn_time2)           # end date
  cat=client.get_events(starttime=t1, endtime=t2, minmagnitude=minmag) #,maxmagnitude=maxmag)
  # stel = row['Elevation']
  stla = row['Latitude']
  stlo = row['Longitude']
  # try:
  for iev, ev in enumerate(cat) :
    # determine index of desired origin type
    ior = 0 
    trlen = 7200
    tbefore = 0
    
    # Get earthquake parameters
    tbeg = ev.origins[ior].time - tbefore
    # df.at[index, 'e_origin_time'] = tbeg

    tend = tbeg + trlen
    evdp = ev.origins[ior].depth
    evla = ev.origins[ior].latitude
    evlo = ev.origins[ior].longitude

    mag = ev.magnitudes[0].mag
    
    # print(ev.origins[ior])
    # Single station - because time range is different
    vals = gps2dist_azimuth(lat1=stla, lon1=stlo, lat2=evla, lon2=evlo)
    dist = vals[0]
    baz = vals[1]
    # az = vals[2]
    gcarc = locations2degrees(lat1=stla, long1=stlo, lat2=evla, long2=evlo)
    # df.at[index, 'e_gcarc'] = gcarc

    if (evdp is not None):
      # get travel times
      model = TauPyModel(model='iasp91')
      arrivals = model.get_travel_times(source_depth_in_km= evdp/1000,
                                        distance_in_degree=gcarc,
                                        phase_list=["P"])
      if len(arrivals):
        # Create a new row
        new_row = {
            'station_name': row['Station'],
            'station_lat': row['Latitude'],
            'station_lon': row['Longitude'],
            'start_time': row['StartTime'],
            'end_time': row['EndTime'],
            'e_origin_time': tbeg,
            'e_lat': evla,
            'e_lon': evlo,
            'e_dep': evdp,
            'e_baz': baz,
            'e_gcarc': gcarc,
            'e_mag': mag,
            'travel_time': tbeg + arrivals[0].time
            }
      
        # df.at[index, 'travel_time'] = row['e_origin_time'] + arrivals[0].time
        new_df = new_df.append(new_row, ignore_index=True)
        # new_df = pd.concat([new_row, new_df.loc[:]]).reset_index(drop=True)
        print("appended")
  # except TypeError:
  #   continue
  # finally:
  

<ipython-input-5-92bcf266df81>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)
<ipython-input-5-92bcf266df81>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)


appended
appended


<ipython-input-5-92bcf266df81>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)


appended


<ipython-input-5-92bcf266df81>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)
<ipython-input-5-92bcf266df81>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)


...

<ipython-input-5-92bcf266df81>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)
<ipython-input-5-92bcf266df81>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)


appended
appended


<ipython-input-5-92bcf266df81>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)
<ipython-input-5-92bcf266df81>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)


appended


<ipython-input-5-92bcf266df81>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)
<ipython-input-5-92bcf266df81>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(new_row, ignore_index=True)


In [ ]:
new_df.to_csv('output_full.csv', index = False)

# END of work /Rough work

In [ ]:
time1 = df['StartTime'].min() # df.loc[0, 'StartTime']
time2 = df.loc[0, 'EndTime']

# Get event catalog
cat_client="IRIS"       # catalog data
client=Client(cat_client)
minmag=6.0 # minimum magnitude

t1=UTCDateTime(time1)           # start date
t2=UTCDateTime(time2)           # end date
cat=client.get_events(starttime=t1, endtime=t2,
                      minmagnitude=minmag) #,maxmagnitude=maxmag)
print(cat)

1676 Event(s) in Catalog:
2023-05-27T00:11:00.980000Z | -18.483, -175.181 | 6.0  mww
2023-05-26T10:03:24.417000Z | +35.518, +140.519 | 6.1  mww
...
2011-08-17T11:44:10.600000Z | +36.788, +143.776 | 6.2  MW
2011-08-16T11:03:57.630000Z |  -2.303, +128.036 | 6.0  MW
To see all events call 'print(CatalogObject.__str__(print_all=True))'


In [ ]:
# Station specific settings
# time1="2015-01-01T00:00:00"     # Start date
# time2="2016-09-01T00:00:00"     # End date

# network= df.loc[0, '#Network']
# station=df.loc[0, 'Station'] # already ran this cell for OJ14
stel= df.loc[0, 'Elevation']
stla= df.loc[0, 'Latitude']
stlo= df.loc[0, 'Longitude']
# Parameters setup
# day_dir = '/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJP_Data/'
# save_dir = '/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJP_Events/'
is_downsamp = 1
sr_new = 1
#invFull=read_inventory('/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJP_Data/xml/PS_'+station+'.xml')

In [ ]:
for iev, ev in enumerate(cat) :  
    # determine index of desired origin type
    ior = 0 
    trlen = 7200
    tbefore = 0
    
    # Get earthquake parameters
    tbeg = ev.origins[ior].time - tbefore
    tend = tbeg + trlen
    evdp = ev.origins[ior].depth
    evla = ev.origins[ior].latitude
    evlo = ev.origins[ior].longitude 
    mag = ev.magnitudes[0].mag
    
    print(ev.origins[ior])
    # extract date for naming folders
    # date = datetime.strptime(str(tbeg),'%Y-%m-%dT%H:%M:%S.%fZ')
    # evname = date.strftime('%Y%m%d%H%M')
    # evdir = save_dir + evname + '/'
    # print(evname)
    
    # if not os.path.exists(evdir):
    #     os.makedirs(evdir)
    #     print('Working on : ',evname,' ,New')
    # else:
    #     print('Working on : ',evname,' ,Existing')
        
    # Single station - because time range is different
    vals = gps2dist_azimuth(lat1=stla, lon1=stlo, lat2=evla, lon2=evlo)
    dist = vals[0]
    baz = vals[1]
    az = vals[2]
    gcarc = locations2degrees(lat1=stla, long1=stlo, lat2=evla, long2=evlo)

    # get travel times
    model = TauPyModel(model='iasp91')
    arrivals = model.get_travel_times(source_depth_in_km= evdp/1000,
                                      distance_in_degree=gcarc,
                                      phase_list=["P"])
    print(arrivals)
    
    # Loop through components
    # comps = ["HHZ","HH1","HH2","HDH"]
    # for icomp, comp in enumerate(comps):   
    #     try:
    #         st = get1cdata(tbeg,trlen,day_dir,station,comp)
    #     except Exception:
    #         print('Missing data for station: ',station)
    #         continue
    #     print(st)
    #     if len(st) > 1: # Check for data gaps and fill with 0's
    #         st.merge(method=1, fill_value=0)
    #     sr = st[0].stats.sampling_rate
        
    #     st.trim(starttime=tbeg, endtime=tend, pad=True, nearest_sample=False, fill_value=0) # make sure correct length
        
    #     st.attach_response(invFull)
    #     sr = st[0].stats.sampling_rate
    #     if comp != "HDH":
    #         st.remove_response(output='DISP', zero_mean=True, taper=True, taper_fraction=0.05, pre_filt=[0.001, 0.005, sr/3, sr/2], water_level=600)  
    #         print('Remove Response Done')
        
    #     st.detrend(type='demean')
    #     st.detrend(type='linear')
    #     st.taper(type="cosine",max_percentage=0.05)

    #     if is_downsamp==1:
    #         st.filter('lowpass', freq=0.4*sr_new, zerophase=True) # anti-alias filter
    #         st.resample(sampling_rate=sr_new)
    #         st.detrend(type='demean')
    #         st.detrend(type='linear')
    #         st.taper(type="cosine",max_percentage=0.05)
        
    #     # convert to SAC and fill out station/event header info
    #     sac = SACTrace.from_obspy_trace(st[0])
    #     sac.b = tbefore
    #     sac.stel = stel
    #     sac.stla = stla
    #     sac.stlo = stlo
    #     sac.evdp = evdp
    #     sac.evla = evla
    #     sac.evlo = evlo
    #     sac.mag = mag
    #     sac.dist = dist
    #     sac.az = az
    #     sac.baz = baz
    #     sac.gcarc = gcarc
        
    #     sac_out = evdir + evname + '.' + network + '.' + station + '.' + comp + '.sac'
    #     sac.write(sac_out)

Origin
	   resource_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?originid=48432939")
	          time: UTCDateTime(2023, 5, 27, 0, 11, 0, 980000)
	     longitude: -175.1814
	      latitude: -18.4833
	         depth: 223908.0
	 creation_info: CreationInfo(author='us,usauto,pt')
0 arrivals
	
Origin
	   resource_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?originid=48429837")
	          time: UTCDateTime(2023, 5, 26, 10, 3, 24, 417000)
	     longitude: 140.519
	      latitude: 35.5179
	         depth: 44528.0
	 creation_info: CreationInfo(author='us,usauto,pt,at')
1 arrivals
	P phase arrival at 589.367 seconds
Origin
	   resource_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?originid=48423193")
	          time: UTCDateTime(2023, 5, 25, 3, 5, 33, 26000)
	     longitude: -77.0881
	      latitude: 8.9283
	         depth: 13000.0
	 creation_info: CreationInfo(author='at,pt,us,usauto')
0 arrivals
	
Origin
	   resource_id: Re

KeyboardInterrupt: ignored

In [ ]:
# for iev, ev in enumerate(cat) :
#     print(ev.origins[1])
# len(cat)
arr = arrivals[0]
print(arr.time)

278.2468933467737


In [ ]:
import obspy
from obspy import read
from obspy.core import UTCDateTime
from numpy import *
from scipy import signal

def get1cdata(ts,td,dir1,sta,cmp):
    ts=UTCDateTime(ts)
    year = ts.year
    julday = str(ts.julday)
    if len(julday)==2:
        julday='0'+julday
    if len(julday)==1:
        julday='00'+julday
    comp12 = True
    # change this (and fc2) accordingly: Original or not?
    fc = f'{dir1}{sta}/{sta}.{year}.{julday}.??.??.??.{cmp}.sac'
    try:
        tc = read(fc)  # read in the file
    except Exception as e:
        print("Failed to read the file")
        print(e)
        return "get1cdata failed"
    
    if ts.hour >= (24-td/60/60):
        end_time = ts + td
        year = end_time.year
        julday = str(end_time.julday)
        if len(julday)==2:
            julday='0'+julday
        if len(julday)==1:
            julday='00'+julday
        # change this (and fc) accordingly: Original or not?
        fc2 = f'{dir1}{sta}/{sta}.{year}.{julday}.??.??.??.{cmp}.sac'
        try:
            tc2 = read(fc2)  # read in the file
        except Exception:
            print("Failed to read file2")
            return "get1cdata failed"
        
        try:
            tc3 = tc + tc2
            tc = tc3.merge()
        except Exception as e:
            print(e)
            return "get1cdata failed"


    return tc.slice(starttime=ts,endtime=(ts+td))
    #end of function